# 2. 딥러닝 학습방법 이해하기

In [ ]:
import numpy as np

## 2.1 신경망을 수식으로 분해

- 지난 시간까지 데이터를 선형 모델로 해석하는 방법을 배웠다.
- 이제부턴 **비선형 모델인 신경망(neural network)**을 배운다.

- 신경망을 수식으로 분해하려면 우선 선형모델을 먼저 이해해야 한다.
- 신경망은 선형 모델과 비선형 함수들의 결합으로 이루어져 있다.

<br>

## 2.2 선형 모델을 수식으로 표현

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1TITqWCPwURPCxtCNAza880ZuvZ09DJxW' width=600/>

- 각 행벡터 $\mathbf{o}_i$ 는 데이터 $\mathbf{x}_i$ 와 가중치 행렬 $W$ 사이의 행렬곱과 절편 $b$ 벡터의 합으로 표현된다고 가정해보자.

$
\qquad
\mathbf{O} = \mathbf{X} \cdot W + \mathbf{b}
$

- $\mathbf{X}$ : 데이터를 모아 놓은 행렬
- $W$ : 행렬 $\mathbf{X}$ 를 다른 공간으로 보내주는 행렬
- $\mathbf{b}$ : y절편에 해당하는 벡터들을 모아 놓은 행렬 (각 행들이 동일한 값을 가짐)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sIsRjv6p2XB4HGuS7kx6VZPGUruYf0Ub' width=600/>

- 데이터가 바뀌면 결과값도 바뀌게 된다.
- 이 때 출력 벡터의 차원은 $d$ 에서 $p$ 로 바뀌게 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1J-C_s95utuau__c0Q0GbISIy-ORqOZaZ' width=400/>

- $d$ 개의 변수로 $p$ 개의 선형 모델을 만들어서 $p$ 개의 잠재변수를 설명하는 모델을 상상해볼 수 있다.
- 위 그림에서 화살표에 해당하는 것이 가중치 행렬의 값들이다.

- 출력 벡터 $\mathbf{O}$ 에 softmax 함수를 합성하면 확률 벡터가 되므로 **특정 클래스 $k$ 에 속할 확률**로 해석할 수 있다.

$$
\text{softmax}(\mathbf{O}) =
\left(
\frac{exp(o_1)}{\sum_{k=1}^p \; exp(o_k)},
\;\cdots,
\;\frac{exp(o_p)}{\sum_{k=1}^p \; exp(o_k)}
\right)
$$

<br>

## cf) 소프트맥스 연산

- 소프트맥스(softmax) 함수는 **모델의 출력을 확률로 해석**할 수 있게 변환해주는 **연산**이다.
- **분류 문제**를 풀 때 선형 모델과 소프트맥스 함수를 결합하여 예측한다.

$
\qquad
\text{softmax}(\mathbf{o}) = \text{softmax}\left( \mathbf{W} \mathbf{x} + \mathbf{b} \right)
$

- softmax 함수를 통해 $\mathbb{R}^p$ 에 있는 벡터를 확률 벡터로 변환할 수 있다.
- ex) `[1, 2, 0]` -> `[0.24, 0.67, 0.09]`

In [ ]:
def softmax(vec):
    denumerator = np.exp(vec - np.max(vec, axis=-1, keepdims=True))
    numerator = np.sum(denumerator, axis=-1, keepdims=True)
    val = denumerator / numerator
    return val

vec = np.array([[1, 2, 0],
                [-1, 0, 1],
                [-10, 0, 10]])
softmax(vec)

array([[2.44728471e-01, 6.65240956e-01, 9.00305732e-02],
       [9.00305732e-02, 2.44728471e-01, 6.65240956e-01],
       [2.06106005e-09, 4.53978686e-05, 9.99954600e-01]])

- 그러나 학습이 아닌 추론을 할 때는 **원-핫(one-hot) 벡터**로 최대값을 가진 주소만 1로 출력하는 연산을 사용해서 softmax 를 사용하진 않는다.

<br>

## 2.3 신경망: 선형 모델 + 활성 함수

- 신경망은 **선형 모델과 활성 함수(activation function)를 합성한 함수**이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1KekLIsde-M_J39ZnvLVxUxdmagkktGAg' width=400/>

$
\qquad
\mathbf{H} = \left( \sigma(\mathbf{z}_1), \dots, \sigma(\mathbf{z}_n) \right)
$

$
\qquad
\sigma(\mathbf{z}) = \sigma \left( \mathbf{Wx + b} \right)
$

<br>

- 활성 함수는 선형 함수의 **출력값 각각의 원소에 개별적으로 적용**된다. (소프트맥스는 출력물 전체에 대해 연산이 이루어짐)
- 그러므로 활성 함수의 출력값은 벡터가 아닌 하나의 실수값을 갖는다.
- 이렇게 변형된 벡터를 **잠재 벡터(뉴런, neuron)**라고 한다.

<br>

- 활성 함수 $\sigma$ 는 비선형함수로 잠재벡터 $\mathbf{z} = (z_1, \dots, z_q)$ 의 각 노드에 개별적으로 적용하여 새로운 잠재벡터 $\mathbf{H} = \left( \sigma(\mathbf{z}_1), \dots, \sigma(\mathbf{z}_n) \right)$ 를 만든다.

<br>

## cf) 활성 함수

- 활성 함수(activation function)는 $\mathbb{R}$ 위에 정의된 **비선형(nonlinear) 함수**로서 딥러닝에서 매우 중요한 개념이다.
- **활성함수를 쓰지 않으면 딥러닝은 선형모형과 차이가 없다.**
- 시그모이드(sigmoid) 함수나 tanh 함수는 전통적으로 많이 쓰이던 활성함수지만 **딥러닝에선 ReLU 함수**를 많이 쓰고 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1PUgHFUP1dY2ZO60-I1Pyh5um1G9WOBy7' width=800/>


<br>

## 2.4 2층(2-layers) 신경망

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=19foBl5qhx4z4bcfF1w4BTF4EvCwbkqmw' width=400/>

$
\qquad
\mathbf{H} = \left( \sigma(\mathbf{z}_1), \dots, \sigma(\mathbf{z}_n) \right)
$

$
\qquad
\sigma(\mathbf{z}) = \sigma \left( \mathbf{W}^{(1)} \mathbf{x} + \mathbf{b}^{(1)} \right)
$

$
\qquad
\mathbf{O} = \mathbf{H} \mathbf{W}^{(2)} + \mathbf{b}^{(2)}
$

<br>

- 잠재벡터 $\mathbf{H}$ 에서 가중치 행렬 $\mathbf{W}^{(2)}$ 와 $\mathbf{b}^{(2)}$ 를 통해 다시 한 번 선형변환해서 출력하게 되면 $\left( \mathbf{W}^{(2)}, \mathbf{W}^{(1)} \right)$ 를 파라미터로 가진 2층(2-layers) 신경망이다.

<br>

## 2.5 다층 퍼셉트론 (MLP, Multi-Layer Perceptron)

- 다층 퍼셉트론(MLP)은 **신경망이 여러 층 합성된 함수**이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1_tpWWWDeBSKiN332s8J7jd6krv96yNG3' width=600/>

<br>

- $\sigma \left( Z^{(1)} \right)$ : $\left( \sigma(z_1), \dots, \sigma(z_n) \right)$ 으로 이루어진 행렬 (각각의 벡터에 적용)
- $H$ 행렬과 $Z$ 행렬의 크기는 동일하다.
- MLP의 파라미터는 $L$ 개의 가중치 행렬 $W^{(L)}, \dots, W^{(1)}$과 $b^{(L)}, \dots, b^{(1)}$ 로 이루어져 있다.
- $l = 1, \dots, L$ 까지 순차적인 신경망 계산을 **순전파(forward propagation)**라 부른다.

<br>

### cf) 층을 여러 개 쌓는 이유

- 이론적으로는 2층 신경망으로도 임의의 연속함수를 근사할 수 있다.
  - 이를 *universal approximation theroem* 이라 부른다.
- 그러나 층이 깊을수록 **목적 함수를 근사하는 데 필요한 뉴런(노드)의 숫자가 훨씬 빨리 줄어들어 좀 더 효율적으로 학습이 가능**하다.
- 층이 얇으면 필요한 뉴런의 숫자가 기하급수적으로 늘어나서 넓은(wide) 신경망이 되어야 한다.

<br>

## 2.6 딥러닝 학습원리: 역전파 알고리즘

- 딥러닝은 **역전파(backpropagation) 알고리즘**을 이용하여 각 층에 사용된 파라미터 $\{ W^{(l)}, b^{(l)} \}_{l=1}^L$ 를 학습한다.


- 경사하강법을 적용해서 각각의 가중치 행렬들을 학습시킬 때 각각의 가중치에 대한 그레디언트 벡터를 계산해야 한다.
- 각 층에 존재하는 파라미터들에 대한 미분을 계산하여 이 미분값을 이용하여 파라미터들을 업데이트할 수 있다.
- 이 파라미터 행렬들의 모든 원소의 갯수 만큼 경사하강법이 적용된다. (선형 모델보다 많은 파라미터에 대해 경사하강법이 적용됨)
- 딥러닝은 여러 층으로 이루어져 있기 때문에 그레디언트 벡터를 한 번에 계산하기 힘들다.
- 역전파 알고리즘을 통해 순차적으로 그레디언트 벡터를 계산하게 된다.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Z5fdwVPhGC4kbZgzPPUYiOHCDCFc7xr-' width=600/>

- 손실 함수를 $\mathcal{L}$ 이라 했을 때 역전파는 $\partial \mathcal{L} / \partial W^{(l)}$ 정보를 계산할 때 사용된다.
  - 손실 함수에 대한 각각의 파라미터에 대한 편미분 값

- 각 층 파라미터의 그래디언트 벡터는 윗층부터 역순으로 계산하게 된다.
- 역전파는 $l = L, \dots, 1$ 순서로 연쇄법칙을 통해 그레디언트 벡터를 전달한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1cUQEXf_uYaDfNpd0_VYF8xy0PVkX_BgG' width=800/>

<br>

## 2.7 역전파 알고리즘 원리 이해하기 (연쇄법칙, chain-rule)

- 역전파 알고리즘은 합성함수 미분법인 **연쇄법칙(chain-rule) 기반 자동미분(auto-differentiation)**을 사용한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1h8rZtO_sZ51sT6RjGmufNNRmaIKZCiOj' width=600/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1uuCjAbzxDWny9Un2d85Xxj_EThvZhQOL' width=600/>

- 연쇄법칙을 통해 합성함수의 미분을 계산할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1ge6zqEcwHNWNOXjAwwHBZ8LwuODmgDHe' width=600/>

- 각 노드의 텐서 값을 컴퓨터가 기억해야 미분 계산이 가능하다. (많은 메모리를 사용)

<br>

## 2.8 예제: 2층 신경망의 역전파 알고리즘

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1-h960MWKqG3cinmU6yKIOHN6p2kIFI5d' width=600/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1xGCDGkWH5z4qD9Mk2y_cCl37URR7BuM5' width=150/>

- $W^{(1)}$ 는 행렬이므로 각 성분에 대한 편미분을 구해야 한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15HhrBVsY6YI4CCZi0j4kFwcC4qiGvBEb' width=800/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1UMxix_ClhEnpKJfHMx5sfKL_BIm199kk' width=800/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1pFekbzALkrA3GJg9FvzpEUz5WCfBJSqv' width=800/>

- $\delta_{rk} \delta_{jk}$ 때문에 $r = k = j$ 만 남는다.